## Imports

In [1]:
import pandas as pd
import numpy as np
import torch
import json

from tqdm import tqdm
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer
from os import walk
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_curve, auc, brier_score_loss

c:\Users\boyan.bogdanov\.conda\envs\llm-detect\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
ROOT_DATA_RAW = '../../data/raw'
HUMAN_JSON_FILE_NAME = 'human.jsonl'
HUMAN_JSON_PATH = f'{ROOT_DATA_RAW}/{HUMAN_JSON_FILE_NAME}'
MODELS_JSON_FOLDER_PATH = f'{ROOT_DATA_RAW}/machines'

In [3]:
BATCH_SIZE = 32
LSTM_UNITS = 256
LSTM_LAYERS = 5
EMBEDDING_SIZE = 512

In [4]:
TEST_SET_FRACTION = 0.3

### Topic grouping 

In [8]:
topic_to_group = {
    'bideninauguration': 'political-events',
    'capitolriot': 'political-events',
    'kamalaharrisvicepresident': 'political-events',
    'trumpimpeachment': 'political-events',
    'twitterbanstrump': 'political-events',
    'colinpowelldead': 'political-events',
    'kylerittenhousenotguilty': 'crime-legal',
    'michiganhighschoolshooting': 'crime-legal',
    'rustmoviesetshooting': 'crime-legal',
    'wyominggabbypetito': 'crime-legal',
    'georgefloydderekchauvin': 'crime-legal',
    'cnnchriscuomo': 'crime-legal',
    'colonialpipelinehack': 'disasters-accidents',
    'hurricaneida': 'disasters-accidents',
    'tigerwoodsaccident': 'disasters-accidents',
    'winterstormtexas': 'disasters-accidents',
    'evergreensuezcanal': 'disasters-accidents',
    'kabulairportattack': 'disasters-accidents',
    'covid19': 'health-economy',
    'stimuluscheck': 'health-economy',
    'citibank500millionmistake': 'health-economy',
    'harryandmeghan': 'social-cultural-tech',
    'tombradysuperbowl': 'social-cultural-tech',
    'facebookoutage': 'social-cultural-tech'
}

In [9]:
def extract_topic_group(id):
    try:
        dash_indices = [i for i, char in enumerate(id) if char == '-']
        second_last_dash_index = dash_indices[-2]
        last_slash_index = id.rindex('/')
        
        # Extract the text between the second-to-last "-" and the last "/"
        topic = id[second_last_dash_index + 1:last_slash_index]
        return topic_to_group[topic]
    except ValueError:
        # In case "-" or "/" are not found, return None
        return None

In [15]:
df = pd.read_json(path_or_buf=HUMAN_JSON_PATH, lines=True)
df['text_index'] = df.index
df['is_llm'] = 0
df['topic_group'] = df['id'].apply(extract_topic_group)

In [16]:
dir_path, dir_names, file_names = next(walk(MODELS_JSON_FOLDER_PATH))

for file_name in file_names:
    temp_df = pd.read_json(path_or_buf=f'{MODELS_JSON_FOLDER_PATH}/{file_name}', lines=True)
    temp_df['text_index'] = temp_df.index
    temp_df['is_llm'] = 1
    temp_df['topic_group'] = temp_df['id'].apply(extract_topic_group)

    df = pd.concat([df, temp_df], ignore_index=True)

df.drop(labels=['id'], inplace=True, axis='columns')

In [19]:
df.tail()
df['topic_group'].value_counts()

topic_group
disasters-accidents     4032
crime-legal             3780
political-events        3682
social-cultural-tech    1988
health-economy          1736
Name: count, dtype: int64

### Tokenize

In [9]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
df['tokenized_text'] = tokenizer(list(df['text'].to_list()))['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors


## Model

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [12]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, layers_num, device, output_size=1, dropout=0):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.output_size= output_size
        self.dropout = dropout
        self.device = device

        self.embed = nn.Embedding(self.vocab_size, self.embedding_size, device=self.device)

        self.lstm = nn.LSTM(
            input_size=self.embedding_size,
            hidden_size=self.hidden_size,
            num_layers=self.layers_num,
            batch_first=True,
            dropout=self.dropout,
            device=self.device
        )

        self.fc = nn.Linear(
            self.hidden_size,
            self.output_size
        )

    def forward(self, X, lengths):
        embeddings = self.embed(X)

        seq_output, (h_n, c_n) = self.lstm(embeddings)

        out = seq_output.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        logits = self.fc(out)
        return logits

## Dataset

In [13]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size

    def __getitem__(self, index):
        return (
            self.X.iloc[index],
            self.y.iloc[index]
        )

In [14]:
def collate_fn(batch):
  # We want to sort the batch by seq length,
  # in order to make the computation more efficient
  batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

  inputs = [torch.LongTensor(x[0]).to(device) for x in batch]
  padded_input = nn.utils.rnn.pad_sequence(inputs, batch_first=True)

  lengths = torch.LongTensor([len(x[0]) for x in batch]).to(device)

  y = torch.FloatTensor(np.array([x[1] for x in batch])).reshape(-1, 1).to(device)

  return padded_input, lengths, y

## Train and test functions

In [15]:
def calculate_accuracy(y_true, y_hat):
    correct_pred = torch.eq(torch.sigmoid(y_hat).round(), y_true).sum().item()
    return (correct_pred / len(y_hat)) * 100

def calculate_f1(y_true, y_hat):
    y_pred = torch.sigmoid(y_hat).round()
    return f1_score(y_true, y_pred)

def calculate_brier(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)
    return brier_score_loss(y_true, y_prob)


def train_step(model, dataloader, loss_fn, optimizer, device):

    model.train()

    train_loss, train_acc = 0, 0
    steps = 0

    for X, lengths, y in dataloader:
        X, y = X.to(device), y.to(device)

        y_hat = model(X, lengths)

        loss = loss_fn(y_hat, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += calculate_accuracy(y_true=y, y_hat=y_hat)
        steps += 1

    return train_loss / steps, train_acc / steps


def test_step(model, dataloader, loss_fn, device):

    model.eval()

    all_y_true = []
    all_y_hat = []

    test_loss = 0
    steps = 0

    with torch.inference_mode():
        for X, lengths, y in dataloader:

            X, y = X.to(device), y.to(device)

            y_hat = model(X, lengths)

            all_y_true.extend(y)
            all_y_hat.extend(y_hat)

            loss = loss_fn(y_hat, y)
            test_loss += loss.item()

            steps += 1

        all_y_true = torch.FloatTensor(all_y_true)
        all_y_hat = torch.FloatTensor(all_y_hat)

        test_accuracy = calculate_accuracy(all_y_true, all_y_hat)
        test_f1 = calculate_f1(all_y_true, all_y_hat)
        test_brier = calculate_brier(all_y_true, all_y_hat)

    return test_loss / steps, test_accuracy, test_f1, test_brier

def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
        "test_f1": [],
        "test_brier": [],
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        end_time = timer()

        test_loss, test_acc, test_f1, test_brier = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        results["test_f1"].append(test_f1)
        results["test_brier"].append(test_brier),

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"test_f1: {test_f1:.4f} | "
            f"test_brier: {test_brier:.4f} | "
            f"time: {(end_time-start_time):.4f}"
        )

    return results

## Train on each LLM dataset separately and test against all others

In [21]:
def test_against_all(model, topic_group, df, loss_fn, device):
    all_results = []

    for other_topic_group in df['topic_group'].unique():
        if other_topic_group == topic_group:
            continue

        other_topic_group_df = df.loc[df['topic_group'] == other_topic_group]
        test_dataset = TextDataset(other_topic_group_df['tokenized_text'], other_topic_group_df['is_llm'])
        test_dataloader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            drop_last=False,
            collate_fn=collate_fn
        )

        results = test_step(
            model,
            test_dataloader,
            loss_fn,
            device
        )

        all_results.append({
            other_topic_group: results
        })

    return all_results

In [17]:
#human_df = df.loc[df['is_llm'] == 0]
#human_train_df, human_test_df = train_test_split(human_df, test_size=TEST_SET_FRACTION, random_state=69)

In [ ]:
final_results = []

for topic_group in df['topic_group'].unique():
    topic_group_df = df.loc[df['topic_group'] == topic_group]
    topic_train_df, topic_test_df = train_test_split(topic_group_df, test_size=TEST_SET_FRACTION, random_state=69)

    train_dataset = TextDataset(topic_train_df['tokenized_text'], topic_train_df['is_llm'])
    test_dataset = TextDataset(topic_test_df['tokenized_text'], topic_test_df['is_llm'])

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=False,
        collate_fn=collate_fn
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=False,
        collate_fn=collate_fn
    )

    model = RNN(tokenizer.vocab_size, EMBEDDING_SIZE, LSTM_UNITS, LSTM_LAYERS, device, dropout=0.6).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00008)
    loss_fn = nn.BCEWithLogitsLoss()

    print(f'Training model for {topic_group}...')

    current_results = train(
        model,
        train_dataloader,
        test_dataloader,
        optimizer,
        loss_fn,
        epochs=4,
        device=device
    )    

    print(f'Finished training model for {topic_group}')

    print(f'Testing against all for {topic_group}...')
    
    results_against_all = test_against_all(
        model,
        topic_group,
        df,
        loss_fn,
        device
    )

    final_results.append({
        'topic_group': topic_group,
        'results_against_itself': current_results,
        'results_against_all': results_against_all
    })

    print(f'Finished testing against all for {topic_group}')

In [135]:
with open('topic-data.json', 'w') as f:
    json.dump(final_results, f)